In [4]:
import os
import torch
import torch.onnx
import numpy as np
import onnx
import onnxruntime
from monai.networks.nets import SegResNet


In [ ]:
# 모델 정의 (예제에 맞게 수정 필요)
model = SegResNet(
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    init_filters=16,
    in_channels=4,
    out_channels=3,
    dropout_prob=0.2,
).cuda()

# 저장된 모델 디렉토리
model_path = "/player/workspace/Python/brain-otock/Model/best_metric_model_0.7747.pth"
onnx_path = "/player/workspace/Python/brain-otock/ONNX/best_metric_model_0.7747.onnx"

# 모델을 평가 모드로 설정
model.eval()

# ONNX 변환 함수
def convert_to_onnx(model, model_path, onnx_path, input_size=(1, 4, 128, 128, 80)):
    # 저장된 모델 불러오기
    model.load_state_dict(torch.load(model_path))
    
    # 더미 입력 생성 (입력 크기: 배치, 채널, 깊이, 높이, 너비)
    dummy_input = torch.randn(*input_size).cuda()
    
    # 모델을 ONNX 형식으로 변환
    torch.onnx.export(
        model,                     # PyTorch 모델
        dummy_input,               # 더미 입력
        onnx_path,                 # 저장할 ONNX 파일 경로
        export_params=True,        # 모델 매개변수를 함께 저장
        opset_version=11,          # ONNX opset 버전
        do_constant_folding=True,  # 상수 폴딩 최적화
        input_names=['input'],     # 입력 텐서 이름
        output_names=['output'],   # 출력 텐서 이름
        dynamic_axes={
            'input': {0: 'batch_size'},    # 배치 크기를 동적으로 처리
            'output': {0: 'batch_size'}
        }
    )
    print(f"Model has been converted to ONNX and saved at {onnx_path}")

# 모델 ONNX로 변환
convert_to_onnx(model, model_path, onnx_path)

In [7]:
# 결과 확인 및 차이 계산 함수
def compare_onnx_pytorch(model, model_path, onnx_path, input_size=(1, 4, 128, 128, 80)):
    # PyTorch 모델 로드
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    # ONNX 모델 로드
    ort_session = onnxruntime.InferenceSession(onnx_path)

    # 더미 입력 생성
    dummy_input = torch.randn(*input_size).cuda()

    # PyTorch 모델 추론
    with torch.no_grad():
        pytorch_output = model(dummy_input)

    # ONNX 모델 추론
    ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.cpu().numpy()}
    ort_outs = ort_session.run(None, ort_inputs)
    onnx_output = torch.tensor(ort_outs[0]).cuda()

    # 출력 값을 직접 비교하여 차이 확인
    difference = torch.abs(pytorch_output - onnx_output)
    max_difference = torch.max(difference).item()
    print(f"Maximum difference between PyTorch and ONNX outputs: {max_difference:.6f}")

In [8]:
compare_onnx_pytorch(model, model_path, onnx_path)

Maximum difference between PyTorch and ONNX outputs: 0.000458
